# Cluster analysis

In this notebook, we will look at how to debug blocked table similarities and the resulting clusters.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [11]:
# Assume that we have the "chartedIn" results
import takco

# tables = takco.HashBag._load('../../output/chartedIn/2-cluster/*.jsonl')
# tables = takco.HashBag._load('../../output/chartedIn/4-coltypes/*.jsonl')
tables = takco.HashBag._load('../../output/chartedIn/7-integrate/*.jsonl')

tables = sorted(tables, key=lambda x: -x.get('numDataRows'))
print(f"Got {sum(1 for _ in tables)} tables")

takco.preview( tables )

Got 69 tables


?,0,1,2,3,4,5,6,7,8,9,10,11
∈,decimal,dateTime,string,Q2031291 Q134556 Q482994,Q43229,string,Q482994 Q4502142 Q2031291,decimal,Q482994 Q208569,Q482994 Q208569,Q2031291 Q134556,decimal
2,,,,,,,,,P361,,,
3,,,,,P2291,P2291,,,,,,
4,,,,,,,,,,P2291,,
6,,,,P2291,,,,,,,,
,Position,Year,Single,Chart,Chart,Chart,_pgTitle,Weeks,Chart,B-Side (or Double A-Side),RIAJ Certification,Sales
,13,2019,,Blinding Lights,Australia ( ARIA ),,,,,,,
,30,2019,,Blinding Lights,Austria ( Ö3 Austria Top 40 ),,,,,,,
,39,2019,,Blinding Lights,Belgium ( Ultratop 50 Flanders),,,,,,,
,1,2019,,Blinding Lights,Belgium ( Ultratip Wallonia),,,,,,,


In [8]:
# Get biggest table
table = tables[0]

from takco.util import tableobj_to_dataframe
df = tableobj_to_dataframe(table)
for ci, col in enumerate(df.columns):
    tophead = table['tableHeaders'][0][ci].get('freq')
    if tophead:
        print("Top headers:",  dict(sorted(tophead.items(), key=lambda x:-x[1])) )
    else:
        print("Header:", col)
    print("Top values:", dict( df.T.iloc[ci].value_counts()[:3] ))
    print()

Top headers: {'Position': 14, 'Peak position': 13, 'Peak Position': 2}
Top values: {'1': 1241, '2': 468, '3': 317}

Top headers: {'Year': 14, '_Variable': 12}
Top values: {'2019': 1317, '2018–19': 457, '': 397}

Top headers: {'_pgTitle': 10, 'Chart': 2, 'Single': 1}
Top values: {'': 5917, 'Womanizer (song)': 34, 'Somebody That I Used to Know': 7}

Top headers: {'Song': 3, 'Single': 3, 'Chart': 3, 'Title': 1, 'Album': 1, 'Name': 1, 'Charts': 1}
Top values: {'': 2245, 'Belgium': 110, 'Netherlands': 99}

Top headers: {'Chart': 7, 'Chart )': 3, 'End of the year chart': 1}
Top values: {'': 5723, 'US Billboard Hot 100': 60, 'ARIA Compilations Chart': 23}

Top headers: {'_pgTitle': 9, 'Chart': 1, 'Chart Albums (': 1}
Top values: {'': 2159, 'Diamonds (Rihanna song)': 112, 'Señorita (Shawn Mendes and Camila Cabello song)': 109}

Top headers: {'Year': 5, '_Variable': 4}
Top values: {'': 5883, '2012': 37, '1972': 24}

Top headers: {'_pgTitle': 6, 'Single': 1}
Top values: {'': 4468, "Stronger (Wha

In [4]:
takco.preview( table.get('partColAligns'), ntables=None )

?,0,1,2,3
∈,string,dateTime,string,string
,_pgTitle,Year,Winner,Category
,Tapestry (Carole King album),1972,Tapestry,Album of the Year
,Tapestry (Carole King album),1972,""" It's Too Late """,Record of the Year
,Tapestry (Carole King album),1972,""" You've Got a Friend """,Song of the Year
,Tapestry (Carole King album),1972,Tapestry,"Best Pop Vocal Performance, Female"
?,0,1,2,3
∈,string,decimal,dateTime,string
,_pgTitle,Position,_Variable,Chart
,Jesus Christ Superstar (album),4,1971,Australian Albums Chart


In [5]:
from IPython.display import display
from takco.cluster import aggregate_similarities
import pandas as pd
sims = pd.read_csv('../../output/chartedIn/5-cluster/sims.csv', index_col=[0,1,2,3]).query('ti1 < ti2')
agg_func = '@mean(  @max(headjacc, @pow(headvec, 4)), @max(bodylsh, bodytype, @pow(bodyvec, 4) ) )'
sims['agg'] = aggregate_similarities(sims, agg_func)

partColAligns = table.get('partColAligns')
ci_head = {}
for p in partColAligns:
    head = [' '.join(c.get('text','') for c in hcol) for hcol in zip(*p['tableHeaders'])]
    l,g = p['partcol_local'], p['partcol_global']
    ci_head.update({ci:(head[ l[pci] ] if head and l else '') for pci,ci in g.items()})
ci_head = pd.Series(ci_head, name='header')

partsims = sims.join(ci_head, on='ci1', how='inner').join(ci_head, how='inner', on='ci2', rsuffix='2')

display(partsims.describe())

# partsims = partsims[partsims.header == '_pgTitle']
# partsims = partsims[partsims.header == 'Region']
# partsims = partsims[partsims['agg'] < .51]
partsims.sort_values('agg', ascending=False).head(60)

/export/scratch1/home/kruit/nary/takco/cluster/cluster.py:108: RuntimeWarning: All-NaN axis encountered
  "max": lambda *args: np.nanmax(args, axis=0),


,headjacc,headvec,bodylsh,bodyvec,bodytype,agg
count,455.000000,427.000000,350.000000,350.000000,949.000000,949.000000
mean,0.104396,0.506191,0.007757,0.652852,0.461538,0.407304
std,0.297905,0.259614,0.048050,0.171803,0.498781,0.431917
min,0.000000,0.000000,0.000000,0.071600,0.000000,0.000000
25%,0.000000,0.385440,0.000000,0.531718,0.000000,0.000011
50%,0.000000,0.495630,0.000000,0.657400,0.000000,0.116301
75%,0.000000,0.684670,0.000000,0.781000,1.000000,1.000000
max,1.000000,1.000000,0.550781,0.995460,1.000000,1.000000


headjacc  headvec   bodylsh  bodyvec  bodytype  agg  \
ti1 ti2 ci1 ci2                                                        
10  95  39  404       NaN      NaN  0.000000  0.43950       1.0  1.0   
29  32  130 140       NaN      NaN       NaN      NaN       1.0  1.0   
    93  131 393       NaN      NaN  0.000000  0.58833       1.0  1.0   
        128 393       NaN      NaN  0.000000  0.81412       1.0  1.0   
22  93  92  393       NaN      NaN  0.000000  0.55311       1.0  1.0   
        89  393       NaN      NaN  0.000000  0.72977       1.0  1.0   
11  93  46  393       NaN      NaN  0.000000  0.57911       1.0  1.0   
        43  393       NaN      NaN  0.000000  0.80895       1.0  1.0   
10  93  42  393       NaN      NaN  0.000000  0.75982       1.0  1.0   
        41  393       NaN      NaN  0.000000  0.89456       1.0  1.0   
        39  393       NaN      NaN  0.000000  0.40087       1.0  1.0   
29  32  129 141       1.0      1.0       NaN      NaN       1.0  1.0   
22  32  90  141       1.0      1.0       NaN      NaN       1.0  1.0   
        91  140       NaN      NaN       NaN      NaN       1.0  1.0   
32  93  139 393       NaN      NaN  0.000000  0.35059       1.0  1.0   
11  32  45  140       NaN      NaN       NaN      NaN       1.0  1.0   
10  32  40  140       1.0      1.0       NaN      NaN       1.0  1.0   
29  32  131 139       1.0      1.0  0.003906  0.86616       1.0  1.0   
        128 139       NaN      NaN  0.000000  0.63216       1.0  1.0   
22  32  89  139       NaN      NaN  0.000000  0.61024       1.0  1.0   
11  32  46  139       1.0      1.0  0.003906  0.86841       1.0  1.0   
        43  139       NaN      NaN  0.000000  0.63859       1.0  1.0   
10  32  39  139       NaN      NaN  0.000000  0.53204       1.0  1.0   
29  32  131 138       NaN      NaN  0.000000  0.53337       1.0  1.0   
        128 138       NaN      NaN  0.000000  0.67857       1.0  1.0   
22  32  92  138       NaN      NaN  0.000000  0.43289       1.0  1.0   
32  93  138 393       NaN      NaN  0.000000  0.48077       1.0  1.0   
50  93  208 393       NaN      NaN  0.000000  0.58239       1.0  1.0   
22  29  90  129       1.0      1.0       NaN      NaN       1.0  1.0   
    93  89  394       NaN      NaN  0.000000  0.60745       1.0  1.0   
11  93  45  396       NaN      NaN       NaN      NaN       1.0  1.0   
10  93  40  396       NaN      NaN       NaN      NaN       1.0  1.0   
50  93  211 395       1.0      1.0       NaN      NaN       1.0  1.0   
32  93  141 395       1.0      1.0       NaN      NaN       1.0  1.0   
29  93  129 395       1.0      1.0       NaN      NaN       1.0  1.0   
22  93  90  395       1.0      1.0       NaN      NaN       1.0  1.0   
60  93  251 394       NaN      NaN  0.000000  0.61462       1.0  1.0   
57  93  239 394       NaN      NaN  0.000000  0.47729       1.0  1.0   
50  93  208 394       NaN      NaN  0.000000  0.50297       1.0  1.0   
32  93  138 394       NaN      NaN  0.000000  0.45606       1.0  1.0   
29  93  128 394       NaN      NaN  0.000000  0.60506       1.0  1.0   
11  93  43  394       NaN      NaN  0.000000  0.62390       1.0  1.0   
50  93  210 393       NaN      NaN  0.000000  0.47485       1.0  1.0   
10  93  39  394       NaN      NaN  0.000000  0.41391       1.0  1.0   
60  93  256 393       NaN      NaN       NaN      NaN       1.0  1.0   
        255 393       NaN      NaN  0.000000  0.46103       1.0  1.0   
        254 393       NaN      NaN  0.000000  0.71875       1.0  1.0   
        253 393       NaN      NaN  0.000000  0.49378       1.0  1.0   
        251 393       NaN      NaN  0.000000  0.69310       1.0  1.0   
57  93  244 393       NaN      NaN       NaN      NaN       1.0  1.0   
        243 393       NaN      NaN  0.000000  0.52192       1.0  1.0   
        242 393       NaN      NaN  0.000000  0.74469       1.0  1.0   
        241 393       NaN      NaN  0.000000  0.52757       1.0  1.0   
        239 393       NaN      NaN  0.000000  0.74350    